In [4]:
import numpy as np
import scipy as sp
from scipy.io import loadmat
import time
import matplotlib.pyplot as plt

In [5]:
#Implementing Jacobi Method
def Jacobi(A,b,x0,tol,numiter):
    n=np.shape(x0)[0]
    x=x0.copy()
    for i in range(0,numiter):
        temp=x.copy()
        for j in range(0,n):
            x[j,0]=(1/A[j,j])*(b[j,0]-A[j,:]@temp+A[j,j]*temp[j,0])
        if(np.linalg.norm(b-A@x)/np.linalg.norm(b)<tol):
            break
        print("\rIteration Status: %d" % (i+1),", Residual Status: %.12E" % np.linalg.norm(b-A@x)/np.linalg.norm(b),end="")
    if(i==numiter-1):
        print('\nMaximum iteration has achieved, without achieving the intended convergence, Residual Status: %.12E' % temp,end="")
    return x,i+1,np.linalg.norm(b-A@x)/np.linalg.norm(b)

#Implementing Gauss-Seidel Method
def GaussSeidel(A,b,x0,tol,numiter):
    n=np.shape(x0)[0]
    x=x0.copy()
    for i in range(0,numiter):
        for j in range(0,n):
            x[j,0]=(1/A[j,j])*(b[j,0]-A[j,:]@x+A[j,j]*x[j,0])
        temp=np.linalg.norm(b-A@x)/np.linalg.norm(b)
        if(temp<tol):
            break
        print("\rIteration Status: %d" % (i+1),", Residual Status: %.12E" % temp,end="")
    if(i==numiter-1):
        print('\nMaximum iteration has achieved, without achieving the intended convergence, Residual Status: %.12E' % temp,end="")
    return x,i+1,temp

#Implementing SOR Method
def SOR(A,b,x0,w,tol,numiter):
    n=np.shape(x0)[0]
    x=x0.copy()
    for i in range(0,numiter):
        for j in range(0,n):
            temp=(1/A[j,j])*(b[j,0]-A[j,:]@x+A[j,j]*x[j,0])
            x[j,0]=w*temp+(1-w)*x[j,0]
        tempr=np.linalg.norm(b-A@x)/np.linalg.norm(b)
        if(tempr<tol):
            break
        print("\rIteration Status: %d" % (i+1),", Residual Status: %.12E" % tempr,end="")
    if(i==numiter-1):
        print('\nMaximum iteration has achieved, without achieving the intended convergence, Residual Status: %.12E' % tempr,end="")
    return x,i+1,tempr

#Implementing Steepest Descent Method
def SD(A,b,x0,tol,numiter):
    n=np.shape(x0)[0]
    x=x0.copy()
    for i in range(0,numiter):
        r=b-A@x
        alpha=(r.T@r)/(r.T@A@r)
        x=x+alpha*r
        temp=np.linalg.norm(b-A@x)/np.linalg.norm(b)
        if(temp<tol):
            break
        print("\rIteration Status: %d" % (i+1),", Residual Status: %.12E" % temp,end="")
    if(i==numiter-1):
        print('\nMaximum iteration has achieved, without achieving the intended convergence, Residual Status: %.12E' % temp,end="")
    return x,i+1,temp

#Implementing Conjugate Gradient Method
def CG(A,b,x0,tol,numiter):
    n=np.shape(x0)[0]
    x=x0.copy()
    r=b-A@x
    p=r.copy()
    for i in range(0,numiter):
        alpha=(r.T@r)/(p.T@A@p)
        x=x+alpha*p
        rnew=r-alpha*A@p
        beta=(rnew.T@rnew)/(r.T@r)
        p=rnew+beta*p
        r=rnew
        temp=np.linalg.norm(b-A@x)/np.linalg.norm(b)
        if(temp<tol):
            break
        print("\rIteration Status: %d" % (i+1),", Residual Status: %.12E" % temp,end="")
    if(i==numiter-1):
        print('\nMaximum iteration has achieved, without achieving the intended convergence, Residual Status: %.12E' % temp,end="")
    return x,i+1,temp

#Implementing Conjugate Gradient Method with Preconditioning: Incomplete Cholesky
def CG_IC(A,b,x0,tol,numiter):
    n=np.shape(x0)[0]
    x=x0.copy()
    r=b-A@x
    L=np.linalg.cholesky(A)
    for i in range(0,n):
        for j in range(0,i):
            if(A[i,j]==0):L[i,j]=0
    z=np.linalg.solve(L,r)
    p=z
    for i in range(0,numiter):
        alpha=(z.T@r)/(p.T@A@p)
        x=x+alpha*p
        rnew=r-alpha*A@p
        znew=np.linalg.solve(L,rnew)
        beta=(rnew.T@znew)/(r.T@z)
        p=znew+beta*p
        r=rnew;z=znew
        temp=np.linalg.norm(r)
        if(temp<tol):break
        print("\rIteration Status: %d" % (i+1),", Residual Status: %.12E" % temp,end="")
    if(i==numiter-1):
        print('\nMaximum iteration has achieved, without achieving the intended convergence, Residual Status: %.12E' % temp,end="")
    return x,i+1,temp

#Implementing Conjugate Gradient Method with Preconditioning: SSOR
def CG_SSOR(A,b,x0,w,tol,numiter):
    n=np.shape(x0)[0]
    x=x0.copy()
    r=b-A@x
    U=np.triu(A);L=np.tril(A);D=np.diag(np.diag(A))
    z=np.linalg.solve(U+(1/w)*D,D@np.linalg.solve(L+(1/w)*D,((2-w)/(w))*r))
    p=z
    for i in range(0,numiter):
        alpha=(z.T@r)/(p.T@A@p)
        x=x+alpha*p
        rnew=r-alpha*A@p
        znew=np.linalg.solve(U+(1/w)*D,D@np.linalg.solve(L+(1/w)*D,((2-w)/(w))*r))
        beta=(rnew.T@znew)/(r.T@z)
        p=znew+beta*p
        r=rnew;z=znew
        temp=np.linalg.norm(r)
        if(temp<tol):break
        print("\rIteration Status: %d" % (i+1),", Residual Status: %.12E" % temp,end="")
    if(i==numiter-1):
        print('\nMaximum iteration has achieved, without achieving the intended convergence, Residual Status: %.12E' % temp,end="")
    return x,i+1,temp

def ARNOLDI(A,x1,m):
    n=np.shape(A)[0]
    Q=np.zeros((n,m+1))
    H=np.zeros((m+1,m))
    Q[:,0]=x1/np.linalg.norm(x1,2)
    for i in range(0,m):
        w=A@Q[:,i]
        for j in range(0,i+1):
            H[j,i]=np.dot(Q[:,j],w)
            w=w-H[j,i]*Q[:,j]
        H[i+1,i]=np.linalg.norm(w,2)
        if H[i+1,i]==0:return Q,H
        Q[:,i+1]=w/H[i+1,i]
    return Q,H

def LANCZOS(A,x0,m):
    n=np.shape(A)[0]
    Q=np.zeros([n,m+2])
    T=np.zeros([m+1,m])
    alpha=np.zeros([m,1])
    beta=np.zeros([m+1,1])
    Q[:,0]=0;beta[0,0]=0;Q[:,1]=x0/np.linalg.norm(x0,2)
    for i in range(1,m+1):
        w=A@Q[:,i]
        alpha[i-1,0]=np.dot(Q[:,i],w)
        w=w-beta[i-1,0]*Q[:,i-1]-alpha[i-1,0]*Q[:,i]
        beta[i,0]=np.linalg.norm(w,2)
        Q[:,i+1]=w/beta[i,0]
    T[0:m,0:m]=np.diag(alpha[0:m,0])+np.diag(beta[1:m,0],-1)+np.diag(beta[1:m,0],1)
    T[m,m-1]=beta[m,0]
    return Q[:,1:m+2],T

def GMRESB(A,b,x0,m,tol,maxiter):
    for i in range(0,maxiter):
        r=b-A@x0
        Q,H=ARNOLDI(A,r,m)
        beta=np.linalg.norm(r,2)
        e1=np.zeros((m+1,1));e1[0,0]=beta
        y=np.linalg.lstsq(H,e1,rcond=None)[0]
        x=x0+Q[:,:m]@y
        r=np.linalg.norm(b-A@x,2)
        if r<tol:
            return x,r,i
        x0=x
        print("\rIteration Status: %d" % (i+1),", Residual Status: %.12E" % r,end="")
        if(i==maxiter-1):
            print('\nMaximum iteration has achieved, without achieving the intended convergence, Residual Status: %.12E' % r,end="")
    return x,r,i

def MINRESB(A,b,x0,m,tol,maxiter):
    for i in range(0,maxiter):
        r=b-A@x0
        Q,T=LANCZOS(A,r,m)
        beta=np.linalg.norm(r,2)
        e1=np.zeros((m+1,1));e1[0,0]=beta
        y=np.linalg.lstsq(T,e1,rcond=None)[0]
        x=x0+Q[:,:m]@y
        r=np.linalg.norm(b-A@x,2)
        if r<tol:
            return x,r,i
        x0=x
        print("\rIteration Status: %d" % (i+1),", Residual Status: %.12E" % r,end="")
        if(i==maxiter-1):
            print('\nMaximum iteration has achieved, without achieving the intended convergence, Residual Status: %.12E' % r,end="")
    return x,r,i

def ILUB(A_val):
    A=A_val.copy()
    n=np.shape(A)[0]
    for i in range(1,n):
        for j in range(0,i):
            if A[i,j]!=0:
                A[i,j]=A[i,j]/A[j,j]
                for k in range(j+1,n):
                    if A[i,k]!=0:
                        A[i,k]=A[i,k]-A[i,j]*A[j,k]
    U=np.triu(A)
    L=np.tril(A,-1)+np.identity(n)
    return L,U

def preGMRESB(A,b,x0,m,tol,maxiter):
    L,U,P=sp.sparse.linalg.spliu(A)
    b_bar=np.linalg.inv(P.T@L)@b
    B=np.linalg.inv(P.T@L)@(A@np.linalg.inv(U))
    x,residue,iteration=GMRESB(B,b_bar,x0,m,tol,maxiter)
    x=np.linalg.inv(U)@x
    r=np.linalg.norm(b-A@x,2)
    return x,r,iteration




In [ ]:
tempPOWERMAT=loadmat('POWERMAT.mat');POWERMAT=np.array(tempPOWERMAT['POWERMAT'])
Si=loadmat('Si.mat');Si=np.array(Si['Si'])
west0479=loadmat('west0479.mat');west0479=np.array(west0479['west0479'])

x,iteration,residue=CG(POWERMAT,np.random.rand(685,1),np.zeros([685,1]),1e-6,100)
